In [1]:
import pandas as pd
import datetime
import os
working_directory = os.getcwd()
print(working_directory)

/Users/vui_tinh/Desktop/Desktop_Stuff/Boston/CS_689/Project


In [2]:
path_1 = working_directory + '/listings_details_amsterdam.csv'
path_2 = working_directory + '/listings_detailed_madrid.csv'
path_3 = working_directory + '/AirBnB_Amsterdam/listings.csv'
path_4 = working_directory + '/AirBnB_Madrid/listings.csv'
path_5 = working_directory + '/AirBnB_Amsterdam/reviews_details.csv'
path_6 = working_directory + '/AirBnB_Madrid/reviews_detailed.csv'
path_7 = working_directory + '/AirBnB_Madrid/calendar.csv'
path_8 = working_directory + '/AirBnB_Amsterdam/calendar.csv'
Amsterdam_df = pd.read_csv(path_1)
Madrid_df  = pd.read_csv(path_2)
Amsterdam_listing_df = pd.read_csv(path_3)
Madrid_listing_df = pd.read_csv(path_4)
Amsterdam_review_df = pd.read_csv(path_5)
Madrid_review_df = pd.read_csv(path_6)
Madrid_Calendar = pd.read_csv(path_7)
Amsterdam_Calendar = pd.read_csv(path_8)

/var/folders/6c/xgblq5v97lb39fths3r32xvm0000gn/T/ipykernel_29711/2130609872.py:9: DtypeWarning: Columns (87) have mixed types. Specify dtype option on import or set low_memory=False.
  Amsterdam_df = pd.read_csv(path_1)


In [6]:
Amsterdam_df.shape

(20030, 96)

In [7]:
Madrid_df.shape

(19618, 74)

In [8]:
Amsterdam_review_df.shape

(431830, 6)

In [9]:
Madrid_review_df.shape

(625006, 6)

In [10]:
Madrid_Calendar.shape

(7161256, 7)

In [11]:
Amsterdam_Calendar.shape

(7310950, 4)

In [5]:
Amsterdam_df.head(5)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,2818,https://www.airbnb.com/rooms/2818,20181206172549,2018-12-06,Quiet Garden View Room & Super Fast WiFi,Quiet Garden View Room & Super Fast WiFi,I'm renting a bedroom (room overlooking the ga...,Quiet Garden View Room & Super Fast WiFi I'm r...,none,"Indische Buurt (""Indies Neighborhood"") is a ne...",...,f,NaN,{Amsterdam},t,f,strict_14_with_grace_period,f,f,1,2.10
1,3209,https://www.airbnb.com/rooms/3209,20181206172549,2018-12-06,"Quiet apt near center, great view",You will love our spacious (90 m2) bright apar...,"Our apartment has lots of light, a balcony and...",You will love our spacious (90 m2) bright apar...,none,Welcome to the Spaarndammerbuurt! From the beg...,...,f,NaN,{Amsterdam},f,f,moderate,f,f,1,1.03
2,20168,https://www.airbnb.com/rooms/20168,20181206172549,2018-12-06,100%Centre-Studio 1 Private Floor/Bathroom,"Cozy studio on your own private floor, 100% in...",For those who like all facets of city life. In...,"Cozy studio on your own private floor, 100% in...",none,Located just in between famous central canals....,...,f,NaN,{Amsterdam},f,f,strict_14_with_grace_period,f,f,2,2.18
3,25428,https://www.airbnb.com/rooms/25428,20181206172549,2018-12-06,Lovely apt in City Centre (Jordaan),NaN,"This nicely furnished, newly renovated apt is...","This nicely furnished, newly renovated apt is...",none,NaN,...,f,NaN,{Amsterdam},f,f,strict_14_with_grace_period,f,f,2,0.09
4,27886,https://www.airbnb.com/rooms/27886,20181206172549,2018-12-06,"Romantic, stylish B&B houseboat in canal district",Stylish and romantic houseboat on fantastic hi...,For a romantic couple: A beautifully restored ...,Stylish and romantic houseboat on fantastic hi...,none,"Central, quiet, safe, clean and beautiful.",...,f,NaN,{Amsterdam},t,f,strict_14_with_grace_period,f,f,1,2.03


In [6]:
Amsterdam_df.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'access', 'interaction', 'house_rules',
       'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url',
       'host_id', 'host_url', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url',
       'host_picture_url', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms',

In [7]:
Amsterdam_review_df.head(2)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2818,1191,2009-03-30,10952,Lam,Daniel is really cool. The place was nice and ...
1,2818,1771,2009-04-24,12798,Alice,Daniel is the most amazing host! His place is ...


In [8]:
Amsterdam_Calendar.head(2)

,listing_id,date,available,price
0,2818,2019-12-05,f,NaN
1,73208,2019-08-30,f,NaN


# I -------Doing for Host Dimension

In [4]:
# Filter columns for Amsterdam host dimension
Host_Amsterdam_df = Amsterdam_df.filter(['host_id','host_name','host_is_superhost','host_response_time'])
Host_Amsterdam_df

,host_id,host_name,host_is_superhost,host_response_time
0,3159,Daniel,t,within an hour
1,3806,Maartje,f,within an hour
2,59484,Alex,f,within a few hours
3,56142,Joan,f,within a few hours
4,97647,Flip,t,within an hour
...,...,...,...,...
20025,13399651,Marieke,f,NaN
20026,1595885,Evita,f,NaN
20027,87866499,Tommaso,f,NaN
20028,1595885,Evita,f,NaN


In [5]:
# Drop duplicates for Host_Amsterdam 
Host_Amsterdam_Dis_df = Host_Amsterdam_df.drop_duplicates()
Host_Amsterdam_Dis_df.head

<bound method NDFrame.head of          host_id host_name host_is_superhost  host_response_time
0           3159    Daniel                 t      within an hour
1           3806   Maartje                 f      within an hour
2          59484      Alex                 f  within a few hours
3          56142      Joan                 f  within a few hours
4          97647      Flip                 t      within an hour
...          ...       ...               ...                 ...
20023  228749822  Rozaliia                 f                 NaN
20024  229361236     Poppy                 f                 NaN
20025   13399651   Marieke                 f                 NaN
20026    1595885     Evita                 f                 NaN
20029   27212057      Lisa                 f                 NaN

[17264 rows x 4 columns]>

In [6]:
# 2. Filter columns for Madrid host dimension
Host_Marid_df = Madrid_df.filter(['host_id','host_name','host_is_superhost','host_response_time'])


In [7]:
# Drop duplicates for Host_Madrid 
Host_Madrid_Dis_df = Host_Marid_df.drop_duplicates()
Host_Madrid_Dis_df

,host_id,host_name,host_is_superhost,host_response_time
0,13660,Simon,t,within an hour
1,83531,Abdel,f,NaN
2,82175,Jesus,f,within an hour
3,346366726,A,f,NaN
4,114340,Agustina,f,NaN
...,...,...,...,...
19593,396428081,Maria,f,NaN
19595,78914217,Zami,f,a few days or more
19599,151861190,Jose,f,NaN
19601,38544397,Juliane,f,within a few hours


In [8]:
# --Combine the two data frame to get host_dimension 

In [9]:
data = [Host_Amsterdam_Dis_df,Host_Madrid_Dis_df]
host_dim_df = pd.concat(data,ignore_index=True)
print(host_dim_df)

         host_id host_name host_is_superhost  host_response_time
0           3159    Daniel                 t      within an hour
1           3806   Maartje                 f      within an hour
2          59484      Alex                 f  within a few hours
3          56142      Joan                 f  within a few hours
4          97647      Flip                 t      within an hour
...          ...       ...               ...                 ...
28584  396428081     Maria                 f                 NaN
28585   78914217      Zami                 f  a few days or more
28586  151861190      Jose                 f                 NaN
28587   38544397   Juliane                 f  within a few hours
28588  269722123         Y                 f                 NaN

[28589 rows x 4 columns]


In [10]:
# Convert null values to meaning values in this dataframe
#host_dim['host_name'] = host_dim['host_name'].fillna('Anomynos')
#host_dim['host_is_superhost'] = host_dim['host_is_superhost'].fillna('f') # logic is no means false
#host_dim['host_response_time'] = host_dim['host_response_time'].fillna('no measures')

# -- host_distinct for loading into DBMS for host_dim

In [11]:
host_distinct = host_dim_df.filter(['host_name','host_is_superhost','host_response_time']).drop_duplicates()
host_distinct['host_id'] = host_distinct.reset_index().index +1
host_distinct

,host_name,host_is_superhost,host_response_time,host_id
0,Daniel,t,within an hour,1
1,Maartje,f,within an hour,2
2,Alex,f,within a few hours,3
3,Joan,f,within a few hours,4
4,Flip,t,within an hour,5
...,...,...,...,...
28581,Kiara,f,NaN,14353
28582,Susana,f,a few days or more,14354
28585,Zami,f,a few days or more,14355
28587,Juliane,f,within a few hours,14356


# II ---------Doing for Listing Dimension

In [12]:
# Filter for Amsterdam Listing Dimension (amsterdam listing staging table)
Amsterdam_listing = Amsterdam_df.filter(['id','name','property_type','room_type','price',
                                        'accommodates','amenities','description'])
Amsterdam_listing.head(5)

,id,name,property_type,room_type,price,accommodates,amenities,description
0,2818,Quiet Garden View Room & Super Fast WiFi,Apartment,Private room,$59.00,2,"{Internet,Wifi,""Paid parking off premises"",""Bu...",Quiet Garden View Room & Super Fast WiFi I'm r...
1,3209,"Quiet apt near center, great view",Apartment,Entire home/apt,$160.00,5,"{Internet,Wifi,Kitchen,""Paid parking off premi...",You will love our spacious (90 m2) bright apar...
2,20168,100%Centre-Studio 1 Private Floor/Bathroom,Townhouse,Entire home/apt,$80.00,2,"{TV,Internet,Wifi,""Paid parking off premises"",...","Cozy studio on your own private floor, 100% in..."
3,25428,Lovely apt in City Centre (Jordaan),Apartment,Entire home/apt,$125.00,3,"{TV,""Cable TV"",Internet,Wifi,Kitchen,Elevator,...","This nicely furnished, newly renovated apt is..."
4,27886,"Romantic, stylish B&B houseboat in canal district",Houseboat,Private room,$150.00,2,"{TV,Internet,Wifi,Breakfast,Heating,""Smoke det...",Stylish and romantic houseboat on fantastic hi...


In [13]:
# Drop Dupliates to get Amsterdam_listing distinct records
Amsterdam_listing_dist = Amsterdam_listing.drop_duplicates()

Amsterdam_listing_dist.head(5)

,id,name,property_type,room_type,price,accommodates,amenities,description
0,2818,Quiet Garden View Room & Super Fast WiFi,Apartment,Private room,$59.00,2,"{Internet,Wifi,""Paid parking off premises"",""Bu...",Quiet Garden View Room & Super Fast WiFi I'm r...
1,3209,"Quiet apt near center, great view",Apartment,Entire home/apt,$160.00,5,"{Internet,Wifi,Kitchen,""Paid parking off premi...",You will love our spacious (90 m2) bright apar...
2,20168,100%Centre-Studio 1 Private Floor/Bathroom,Townhouse,Entire home/apt,$80.00,2,"{TV,Internet,Wifi,""Paid parking off premises"",...","Cozy studio on your own private floor, 100% in..."
3,25428,Lovely apt in City Centre (Jordaan),Apartment,Entire home/apt,$125.00,3,"{TV,""Cable TV"",Internet,Wifi,Kitchen,Elevator,...","This nicely furnished, newly renovated apt is..."
4,27886,"Romantic, stylish B&B houseboat in canal district",Houseboat,Private room,$150.00,2,"{TV,Internet,Wifi,Breakfast,Heating,""Smoke det...",Stylish and romantic houseboat on fantastic hi...


In [14]:
# Filter for Madrid Listing Dimension (Madrid staging table)
Madrid_listing = Madrid_df.filter(['id','name','property_type','room_type','price',
                                   'accommodates','amenities','description'])
Madrid_listing.head(5)

,id,name,property_type,room_type,price,accommodates,amenities,description
0,6369,"Rooftop terrace room , ensuite bathroom",Private room in apartment,Private room,$60.00,2,"[""Kitchen"", ""Elevator"", ""Extra pillows and bla...",Excellent connection with the AIRPORT and EXHI...
1,21853,Bright and airy room,Private room in apartment,Private room,$31.00,1,"[""Bed linens"", ""Refrigerator"", ""Dishes and sil...",We have a quiet and sunny room with a good vie...
2,23001,Apartmento Arganzuela- Madrid Rio,Entire apartment,Entire home/apt,$50.00,6,"[""Pool"", ""Dishwasher"", ""Bed linens"", ""Refriger...","Apartamento de tres dormitorios dobles, gran s..."
3,24805,Gran Via Studio Madrid,Entire apartment,Entire home/apt,$92.00,3,"[""Refrigerator"", ""Host greets you"", ""Dishes an...","Studio located 50 meters from Gran Via, next t..."
4,26825,Single Room whith private Bathroom,Private room in house,Private room,$26.00,1,"[""Elevator"", ""Patio or balcony"", ""Bed linens"",...",Nice and cozy roon for one person with a priva...


In [15]:
# Drop Dupliates to get Amsterdam_listing distinct records
Madrid_listing_dist = Madrid_listing.drop_duplicates()

Madrid_listing_dist.head(5)

,id,name,property_type,room_type,price,accommodates,amenities,description
0,6369,"Rooftop terrace room , ensuite bathroom",Private room in apartment,Private room,$60.00,2,"[""Kitchen"", ""Elevator"", ""Extra pillows and bla...",Excellent connection with the AIRPORT and EXHI...
1,21853,Bright and airy room,Private room in apartment,Private room,$31.00,1,"[""Bed linens"", ""Refrigerator"", ""Dishes and sil...",We have a quiet and sunny room with a good vie...
2,23001,Apartmento Arganzuela- Madrid Rio,Entire apartment,Entire home/apt,$50.00,6,"[""Pool"", ""Dishwasher"", ""Bed linens"", ""Refriger...","Apartamento de tres dormitorios dobles, gran s..."
3,24805,Gran Via Studio Madrid,Entire apartment,Entire home/apt,$92.00,3,"[""Refrigerator"", ""Host greets you"", ""Dishes an...","Studio located 50 meters from Gran Via, next t..."
4,26825,Single Room whith private Bathroom,Private room in house,Private room,$26.00,1,"[""Elevator"", ""Patio or balcony"", ""Bed linens"",...",Nice and cozy roon for one person with a priva...


In [16]:
## ------LISTING-DIM--Combine the two distinct data frame (Amsterdam_listing_dist and Madrid_listing_dist)
data1 = [Amsterdam_listing_dist,Madrid_listing_dist]

Listing_Distinct = pd.concat(data1, ignore_index = True)

In [17]:
# Add effective_timestamp, Expire_timestamp and current_flag for listing_Dim
Listing_Distinct['effective_timestamp'] = pd.Timestamp.today().strftime('%Y-%m-%d')
Listing_Distinct['expire_timestamp'] = datetime.date(9999, 12, 31)
Listing_Distinct['current_flag'] = 'Current'
Listing_Distinct.head(2)

,id,name,property_type,room_type,price,accommodates,amenities,description,effective_timestamp,expire_timestamp,current_flag
0,2818,Quiet Garden View Room & Super Fast WiFi,Apartment,Private room,$59.00,2,"{Internet,Wifi,""Paid parking off premises"",""Bu...",Quiet Garden View Room & Super Fast WiFi I'm r...,2023-04-15,9999-12-31,Current
1,3209,"Quiet apt near center, great view",Apartment,Entire home/apt,$160.00,5,"{Internet,Wifi,Kitchen,""Paid parking off premi...",You will love our spacious (90 m2) bright apar...,2023-04-15,9999-12-31,Current


In [18]:
# Create listing dimension key
Listing_Distinct['listing_dim_id'] = Listing_Distinct.reset_index().index + 1

In [19]:
# Transform price in the listing_distinct
Listing_Distinct['price'] = Listing_Distinct['price'].str.extract('(\d+)').astype(int)

# 2---This is listing dimension ready to load into dbms

In [20]:
Listing_Distinct.head(5) 

,id,name,property_type,room_type,price,accommodates,amenities,description,effective_timestamp,expire_timestamp,current_flag,listing_dim_id
0,2818,Quiet Garden View Room & Super Fast WiFi,Apartment,Private room,59,2,"{Internet,Wifi,""Paid parking off premises"",""Bu...",Quiet Garden View Room & Super Fast WiFi I'm r...,2023-04-15,9999-12-31,Current,1
1,3209,"Quiet apt near center, great view",Apartment,Entire home/apt,160,5,"{Internet,Wifi,Kitchen,""Paid parking off premi...",You will love our spacious (90 m2) bright apar...,2023-04-15,9999-12-31,Current,2
2,20168,100%Centre-Studio 1 Private Floor/Bathroom,Townhouse,Entire home/apt,80,2,"{TV,Internet,Wifi,""Paid parking off premises"",...","Cozy studio on your own private floor, 100% in...",2023-04-15,9999-12-31,Current,3
3,25428,Lovely apt in City Centre (Jordaan),Apartment,Entire home/apt,125,3,"{TV,""Cable TV"",Internet,Wifi,Kitchen,Elevator,...","This nicely furnished, newly renovated apt is...",2023-04-15,9999-12-31,Current,4
4,27886,"Romantic, stylish B&B houseboat in canal district",Houseboat,Private room,150,2,"{TV,Internet,Wifi,Breakfast,Heating,""Smoke det...",Stylish and romantic houseboat on fantastic hi...,2023-04-15,9999-12-31,Current,5


# --- Concate amsterdam and madrid listing staging tables

In [3]:

listing_data_df = [Amsterdam_df ,Madrid_df] 
Amsterdame_Madrid_staging = pd.concat(listing_data_df, ignore_index = True)

In [22]:
# Transform price
Amsterdame_Madrid_staging['price'] = Amsterdame_Madrid_staging['price'].str.extract('(\d+)').astype(int)

In [4]:
Amsterdame_Madrid_staging.shape

(39648, 108)

# III-------Doing Reviewer dimension

In [24]:

#--Amsterdam
Amsterdam_reviewer_df = Amsterdam_review_df.filter(['reviewer_id', 'reviewer_name','comments'])
Amsterdam_reviewer_dist = Amsterdam_reviewer_df.drop_duplicates()

In [25]:
Amsterdam_reviewer_dist.head(5)

,reviewer_id,reviewer_name,comments
0,10952,Lam,Daniel is really cool. The place was nice and ...
1,12798,Alice,Daniel is the most amazing host! His place is ...
2,11869,Natalja,We had such a great time in Amsterdam. Daniel ...
3,14064,Enrique,Very professional operation. Room is very clea...
4,17977,Sherwin,Daniel is highly recommended. He provided all...


In [26]:
#--Madrid
Madrid_reviewer_df = Madrid_review_df.filter(['reviewer_id', 'reviewer_name','comments'])
Madrid_reviewer_dist = Madrid_reviewer_df.drop_duplicates()
Madrid_reviewer_dist.head(5)

,reviewer_id,reviewer_name,comments
0,84790,Nancy,Simon and Arturo have the ultimate location in...
1,84338,David,Myself and Kristy originally planned on stayin...
2,98655,Marion,We had a great time at Arturo and Simon's ! A ...
3,109871,Kurt,I very much enjoyed the stay. \r\nIt's a wond...
4,98901,Dennis,Arturo and Simon are polite and friendly hosts...


# ---Reviewer-Dim-----Combine Amsterdam_reviewer_dist and Madrid_reviewer_dist to get into Reviewer Dimension

In [27]:

data_reviewer = [Amsterdam_reviewer_dist,Madrid_reviewer_dist]
reviewer_dim = pd.concat(data_reviewer,ignore_index=True)
reviewer_dim.head(5)

,reviewer_id,reviewer_name,comments
0,10952,Lam,Daniel is really cool. The place was nice and ...
1,12798,Alice,Daniel is the most amazing host! His place is ...
2,11869,Natalja,We had such a great time in Amsterdam. Daniel ...
3,14064,Enrique,Very professional operation. Room is very clea...
4,17977,Sherwin,Daniel is highly recommended. He provided all...


In [28]:
# add new surrogate id as reviewer_dim_id
reviewer_dim['reviewer_dim_id'] = reviewer_dim.reset_index().index + 1

# 3. This is reviewer dimension

In [29]:
reviewer_dim

,reviewer_id,reviewer_name,comments,reviewer_dim_id
0,10952,Lam,Daniel is really cool. The place was nice and ...,1
1,12798,Alice,Daniel is the most amazing host! His place is ...,2
2,11869,Natalja,We had such a great time in Amsterdam. Daniel ...,3
3,14064,Enrique,Very professional operation. Room is very clea...,4
4,17977,Sherwin,Daniel is highly recommended. He provided all...,5
...,...,...,...,...
1056261,133082364,Neus,"Ubicación genial, en el maravilloso barrio de ...",1056262
1056262,104936627,Irene,Muy buena atención y alojamiento muy cómodo.,1056263
1056263,196731765,Teodoro,muy buena comunicación y limpieza,1056264
1056264,266607018,Gerson,great spot great location jorge is very attentive,1056265


# ---Combine amsterdam_review_df and madrid_review_df as staging table

In [30]:

review_data_df = [Amsterdam_review_df ,Madrid_review_df ]
Amsterdame_Madrid_review_staging = pd.concat(review_data_df, ignore_index = True)

In [31]:
Amsterdame_Madrid_review_staging

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2818,1191,2009-03-30,10952,Lam,Daniel is really cool. The place was nice and ...
1,2818,1771,2009-04-24,12798,Alice,Daniel is the most amazing host! His place is ...
2,2818,1989,2009-05-03,11869,Natalja,We had such a great time in Amsterdam. Daniel ...
3,2818,2797,2009-05-18,14064,Enrique,Very professional operation. Room is very clea...
4,2818,3151,2009-05-25,17977,Sherwin,Daniel is highly recommended. He provided all...
...,...,...,...,...,...,...
1056831,48990543,748625626,2021-04-14,133082364,Neus,"Ubicación genial, en el maravilloso barrio de ..."
1056832,48991692,745277975,2021-04-04,104936627,Irene,Muy buena atención y alojamiento muy cómodo.
1056833,48992883,747729871,2021-04-11,196731765,Teodoro,muy buena comunicación y limpieza
1056834,48992883,748199100,2021-04-12,266607018,Gerson,great spot great location jorge is very attentive


In [32]:
Amsterdam_review_df

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2818,1191,2009-03-30,10952,Lam,Daniel is really cool. The place was nice and ...
1,2818,1771,2009-04-24,12798,Alice,Daniel is the most amazing host! His place is ...
2,2818,1989,2009-05-03,11869,Natalja,We had such a great time in Amsterdam. Daniel ...
3,2818,2797,2009-05-18,14064,Enrique,Very professional operation. Room is very clea...
4,2818,3151,2009-05-25,17977,Sherwin,Daniel is highly recommended. He provided all...
...,...,...,...,...,...,...
431825,30435529,355474875,2018-12-04,131376847,Andy,The host canceled this reservation the day bef...
431826,30439111,355041813,2018-12-02,103201416,Mohammed,The host canceled this reservation 20 days bef...
431827,30518120,355519016,2018-12-04,228738702,Valeriya,The host canceled this reservation 22 days bef...
431828,30525429,355658538,2018-12-05,111439550,Lucas,I loved this house! It is truly incredible and...


# IV --Date Dimension

In [33]:

path_9 = working_directory + '/date_sample_k.csv'
dat_df = pd.read_csv(path_9)

In [34]:
path_10 = working_directory + '/2008-2013.csv'
dat_df2 = pd.read_csv(path_10)

In [35]:
path_11 = working_directory + '/2014-2019.csv'
dat_df3 = pd.read_csv(path_11)

In [36]:
frame = [dat_df2,dat_df3,dat_df]
date_staging = pd.concat(frame,ignore_index = True)


In [37]:
date_staging.columns

Index(['date key', 'full date', 'day of week', 'day num in month',
       'day num overall', 'day name', 'day abbrev', 'weekday flag',
       'week num in year', 'week num overall', 'week begin date',
       'week begin date key', 'month', 'month num overall', 'month name',
       'month abbrev', 'quarter', 'year', 'yearmo', 'fiscal month',
       'fiscal quarter', 'fiscal year', 'month end flag', 'same day year ago'],
      dtype='object')

In [38]:
date_dimension = date_staging.filter(['date key', 'full date', 'day num in month','month name','year']).drop_duplicates()
date_dimension_dist = date_dimension.rename(columns = {'date key':'time_id', 'full date': 'date', 
                                                       'day num in month':'day',
                                                       'month name': 'month'})

# 4. Date_Dimension distinct

In [39]:

date_dimension_dist

,time_id,date,day,month,year
0,20080101,2008-01-01,1,January,2008
1,20080102,2008-01-02,2,January,2008
2,20080103,2008-01-03,3,January,2008
3,20080104,2008-01-04,4,January,2008
4,20080105,2008-01-05,5,January,2008
...,...,...,...,...,...
6570,20231227,2023-12-27,27,December,2023
6571,20231228,2023-12-28,28,December,2023
6572,20231229,2023-12-29,29,December,2023
6573,20231230,2023-12-30,30,December,2023


# V -- Doing Property_available dimension

In [40]:
property_char = Amsterdame_Madrid_staging.filter(['id','minimum_nights','maximum_nights','instant_bookable'])

In [41]:
property_char.isnull().sum()

id                  0
minimum_nights      0
maximum_nights      0
instant_bookable    0
dtype: int64

In [42]:
# property listing and some attributes (minimum_nights, maximum_nights, instant_bookable)
property_char_df = property_char.drop_duplicates()

In [43]:
property_char_df

,id,minimum_nights,maximum_nights,instant_bookable
0,2818,3,15,t
1,3209,4,20,f
2,20168,1,1000,f
3,25428,14,60,f
4,27886,2,730,t
...,...,...,...,...
39643,49185822,30,1125,f
39644,49186179,30,1125,f
39645,49187258,7,1125,f
39646,49187471,30,1125,f


In [44]:
# getting available attributes

In [45]:
Madrid_available = Madrid_Calendar.filter(['listing_id','date','available'])
Madrid_available_df = Madrid_available.drop_duplicates()

In [46]:
Amsterdam_available = Amsterdam_Calendar.filter(['listing_id','date','available'])
Amsterdam_available_df = Amsterdam_available.drop_duplicates()

In [47]:
# Combine Amsterdam_available_df and Madrid_available_df to get Combine_listing_available

In [48]:
available_data = [Madrid_available_df,Amsterdam_available_df]
Combine_listing_available = pd.concat(available_data, ignore_index = True).rename(columns = {'listing_id': 'id'})
Combine_listing_available

,id,date,available
0,167183,2021-04-15,f
1,6369,2021-04-15,t
2,6369,2021-04-16,t
3,6369,2021-04-17,t
4,6369,2021-04-18,t
...,...,...,...
14472201,29979667,2018-12-11,t
14472202,29979667,2018-12-10,t
14472203,29979667,2018-12-09,t
14472204,29979667,2018-12-08,t


In [49]:
Combine_listing_available

,id,date,available
0,167183,2021-04-15,f
1,6369,2021-04-15,t
2,6369,2021-04-16,t
3,6369,2021-04-17,t
4,6369,2021-04-18,t
...,...,...,...
14472201,29979667,2018-12-11,t
14472202,29979667,2018-12-10,t
14472203,29979667,2018-12-09,t
14472204,29979667,2018-12-08,t


# 5 --- property_available_dim for loading into property_available dimension

In [80]:
property_available_df1 = Combine_listing_available.merge(property_char_df, how = 'inner', on = ['id'])
property_available_df1

,id,date,available,minimum_nights,maximum_nights,instant_bookable
0,167183,2021-04-15,f,1,5,f
1,167183,2021-04-16,f,1,5,f
2,167183,2021-04-17,t,1,5,f
3,167183,2021-04-18,t,1,5,f
4,167183,2021-04-19,t,1,5,f
...,...,...,...,...,...,...
14472201,29979667,2018-12-11,t,1,1125,t
14472202,29979667,2018-12-10,t,1,1125,t
14472203,29979667,2018-12-09,t,1,1125,t
14472204,29979667,2018-12-08,t,1,1125,t


In [83]:
property_available_dim = property_available_df1.filter(['available','minimum_nights','maximum_nights','instant_bookable']).drop_duplicates()


In [84]:
property_available_dim['new_id'] = dd.reset_index().index + 1

In [88]:
# use to load into DBMS
property_available_dim.shape

(2973, 5)

In [89]:
# use property_stage for combining to do booking fact table below
property_stage = property_available_df1.merge(property_available_dim, how = 'inner', 
                                 on = ['available','minimum_nights','maximum_nights','instant_bookable'])

property_stage


,id,date,available,minimum_nights,maximum_nights,instant_bookable,new_id
0,167183,2021-04-15,f,1,5,f,1
1,167183,2021-04-16,f,1,5,f,1
2,2797087,2021-04-16,f,1,5,f,1
3,2797087,2021-04-17,f,1,5,f,1
4,2797087,2021-07-15,f,1,5,f,1
...,...,...,...,...,...,...,...
14472201,30576148,2018-12-29,t,7,12,f,2973
14472202,30576148,2018-12-28,t,7,12,f,2973
14472203,30576148,2018-12-27,t,7,12,f,2973
14472204,30576148,2018-12-26,t,7,12,f,2973


# VI -- Doing Location Dimension

In [54]:
# Amsterdam location

In [55]:
Amsterdam_location_df = Amsterdam_df.filter(['id', 'country', 'city', 
        'neighbourhood_cleansed']).rename(columns = {'neighbourhood_cleansed' : 'neighbourhood'}).drop_duplicates()
Amsterdam_location_df

,id,country,city,neighbourhood
0,2818,Netherlands,Amsterdam,Oostelijk Havengebied - Indische Buurt
1,3209,Netherlands,Amsterdam,Westerpark
2,20168,Netherlands,Amsterdam,Centrum-Oost
3,25428,Netherlands,Amsterdam,Centrum-West
4,27886,Netherlands,Amsterdam,Centrum-West
...,...,...,...,...
20025,30576148,Netherlands,Amsterdam,Watergraafsmeer
20026,30577727,Netherlands,Amsterdam,Oostelijk Havengebied - Indische Buurt
20027,30578037,Netherlands,Amsterdam,Oud-Oost
20028,30579673,Netherlands,Amsterdam,Oostelijk Havengebied - Indische Buurt


In [56]:
Amsterdam_location_df.isnull().sum()

id               0
country          0
city             4
neighbourhood    0
dtype: int64

In [57]:
# Fill NaN with Amsterdam in city columns
Amsterdam_location_df['city'] = Amsterdam_location_df['city'].fillna('Amsterdam')

In [58]:
Amsterdam_location_df

,id,country,city,neighbourhood
0,2818,Netherlands,Amsterdam,Oostelijk Havengebied - Indische Buurt
1,3209,Netherlands,Amsterdam,Westerpark
2,20168,Netherlands,Amsterdam,Centrum-Oost
3,25428,Netherlands,Amsterdam,Centrum-West
4,27886,Netherlands,Amsterdam,Centrum-West
...,...,...,...,...
20025,30576148,Netherlands,Amsterdam,Watergraafsmeer
20026,30577727,Netherlands,Amsterdam,Oostelijk Havengebied - Indische Buurt
20027,30578037,Netherlands,Amsterdam,Oud-Oost
20028,30579673,Netherlands,Amsterdam,Oostelijk Havengebied - Indische Buurt


In [59]:
# Madrid location

In [60]:
Madrid_location = Madrid_df.filter(['id','host_location','neighbourhood_cleansed'])

In [61]:
Madrid_location

,id,host_location,neighbourhood_cleansed
0,6369,"Madrid, Madrid, Spain",Hispanoamérica
1,21853,"Madrid, Madrid, Spain",Cármenes
2,23001,"Madrid, Community of Madrid, Spain",Legazpi
3,24805,ES,Universidad
4,26825,"Madrid, Madrid, Spain",Legazpi
...,...,...,...
19613,49185822,"Madrid, Community of Madrid, Spain",Embajadores
19614,49186179,"Madrid, Community of Madrid, Spain",Gaztambide
19615,49187258,"Madrid, Community of Madrid, Spain",Arapiles
19616,49187471,"Madrid, Community of Madrid, Spain",Guindalera


In [62]:
# Airbnb Madrid datasets do not have country, city columns, so I need to transform from host_location column

Madrid_location['city'] = Madrid_location['host_location'].str.extract(r'(\bMadrid\b)')
Madrid_location['country'] = Madrid_location['host_location'].str.extract(r'(\bSpain\b)')


In [63]:
Madrid_location.shape

(19618, 5)

In [64]:
# Fill NaN values for city and country o Madrid_location
Madrid_location['city'] = Madrid_location['city'].fillna('Madrid')
Madrid_location['country'] = Madrid_location['country'].fillna('Spain')

In [65]:
Madrid_location_df = Madrid_location.filter(['id','neighbourhood_cleansed', 'city',
                    'country']).rename(columns = {'neighbourhood_cleansed':'neighbourhood'}).drop_duplicates()
Madrid_location_df = Madrid_location_df.filter(['id','country','city','neighbourhood'])

In [66]:
Madrid_location_df

,id,country,city,neighbourhood
0,6369,Spain,Madrid,Hispanoamérica
1,21853,Spain,Madrid,Cármenes
2,23001,Spain,Madrid,Legazpi
3,24805,Spain,Madrid,Universidad
4,26825,Spain,Madrid,Legazpi
...,...,...,...,...
19613,49185822,Spain,Madrid,Embajadores
19614,49186179,Spain,Madrid,Gaztambide
19615,49187258,Spain,Madrid,Arapiles
19616,49187471,Spain,Madrid,Guindalera


In [67]:
# --- Combine Amsterdam_location_df and Madrid_location_df to get location_df 

In [68]:
location_data = [Amsterdam_location_df,Madrid_location_df]
location_df = pd.concat(location_data,ignore_index = True)
location_df

,id,country,city,neighbourhood
0,2818,Netherlands,Amsterdam,Oostelijk Havengebied - Indische Buurt
1,3209,Netherlands,Amsterdam,Westerpark
2,20168,Netherlands,Amsterdam,Centrum-Oost
3,25428,Netherlands,Amsterdam,Centrum-West
4,27886,Netherlands,Amsterdam,Centrum-West
...,...,...,...,...
39643,49185822,Spain,Madrid,Embajadores
39644,49186179,Spain,Madrid,Gaztambide
39645,49187258,Spain,Madrid,Arapiles
39646,49187471,Spain,Madrid,Guindalera


# --5--location_distinct_df for loading data into location_dim in DBMS

In [69]:
location_distinct_df = location_df.filter(['country','city','neighbourhood']).drop_duplicates()

In [70]:
location_distinct_df

,country,city,neighbourhood
0,Netherlands,Amsterdam,Oostelijk Havengebied - Indische Buurt
1,Netherlands,Amsterdam,Westerpark
2,Netherlands,Amsterdam,Centrum-Oost
3,Netherlands,Amsterdam,Centrum-West
5,Netherlands,Amsterdam,Bos en Lommer
...,...,...,...
24680,Spain,Madrid,Corralejos
25294,Spain,Madrid,Pradolongo
25694,Spain,Madrid,Pavones
30779,Spain,Madrid,Horcajo


# 1.1 Listing Dimension

In [112]:

Listing_Distinct_load.shape

(39648, 12)

In [ ]:
# Transform price in the Amsterdame_Madrid_staging
#Amsterdame_Madrid_staging['price'] = Amsterdame_Madrid_staging['price'].str.extract('(\d+)').astype(int)

# Join back listing dimension to amsterdam and madrid listing staging table

In [ ]:

review_fact_1 = Amsterdame_Madrid_staging.merge(Listing_Distinct,how = 'left', on =
            ['id','name','property_type','room_type','price', 'accommodates','amenities','description'])

In [ ]:
review_fact_1_listing = review_fact_1.rename(columns = {'id': 'listing_id'})


In [ ]:
review_fact_1_listing.shape

# --1.2 Reviewer dimension

In [ ]:
reviewer_dim.head(5)

# join back to the review staging table

In [ ]:

review_fact_2_r = Amsterdame_Madrid_review_staging.merge(reviewer_dim, how = 'inner', on = ['reviewer_id',
                                                                            'reviewer_name', 'comments'])

In [ ]:
review_fact_2_r.shape

# 1.3 -Date Dimension

In [ ]:
date_dimension_dist.head(5)

# Merge review_fact_2_r with date dimension

In [ ]:

combine_review_fact_2_date = review_fact_2_r.merge(date_dimension_dist, how = 'inner', on = ['date'])


# ----Joining Amsterdam_Madrid Staging table, date dimension, review staging table

In [ ]:
a = combine_review_fact_2_date.query("listing_id == 42713110") # some listing has reviewer review but does not have listing


In [ ]:
a.head(2)

In [ ]:
b = review_fact_1_listing.query("listing_id ==1488680")
b

In [ ]:
Review_fact_2 = combine_review_fact_2_date.merge(review_fact_1_listing, how = 'inner', on = 'listing_id')

In [ ]:
####----Create Review_fact surrogate key
Review_fact_2['review_dim_id'] = Review_fact_2.reset_index().index + 1

In [ ]:
Review_fact_2

# Review Fact load data frame to load into DBMS

In [ ]:
Review_fact_load_2 = Review_fact_2.filter(['review_dim_id','time_id','listing_dim_id','reviewer_dim_id',
                                     'review_scores_accuracy','review_scores_communication','review_scores_rating'])

In [ ]:
Review_fact_load_2

In [ ]:
Review_fact_load_21 = Review_fact_load_2.fillna(0)

# Data for Booking Fact Table

In [90]:
df1 = property_stage.merge(date_dimension_dist, how = 'inner', on = ['date'])

In [91]:
df = Amsterdame_Madrid_staging.filter(['id','host_name','host_is_superhost','host_response_time'])

In [92]:
df2 = df1.merge(location_df, how = 'inner', on = ['id'])


In [93]:
df3 = df2.merge(df, how = 'inner', on = ['id'])

In [94]:
df3.shape

(14472206, 17)

In [95]:
df4 = df3.merge(host_distinct, how = 'inner', on = ['host_name','host_is_superhost','host_response_time'])


In [97]:
df4

,id,date,available,minimum_nights,maximum_nights,instant_bookable,new_id,time_id,day,month,year,country,city,neighbourhood,host_name,host_is_superhost,host_response_time,host_id
0,167183,2021-04-15,f,1,5,f,1,20210415,15,April,2021,Spain,Madrid,Piovera,Consuelo,f,a few days or more,9662
1,167183,2021-04-16,f,1,5,f,1,20210416,16,April,2021,Spain,Madrid,Piovera,Consuelo,f,a few days or more,9662
2,167183,2021-04-17,t,1,5,f,2,20210417,17,April,2021,Spain,Madrid,Piovera,Consuelo,f,a few days or more,9662
3,167183,2021-07-15,t,1,5,f,2,20210715,15,July,2021,Spain,Madrid,Piovera,Consuelo,f,a few days or more,9662
4,167183,2021-07-16,t,1,5,f,2,20210716,16,July,2021,Spain,Madrid,Piovera,Consuelo,f,a few days or more,9662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14472201,30468760,2018-12-10,f,15,21,f,2969,20181210,10,December,2018,Netherlands,Amsterdam,De Baarsjes - Oud-West,Nawid,f,NaN,9609
14472202,30468760,2018-12-09,f,15,21,f,2969,20181209,9,December,2018,Netherlands,Amsterdam,De Baarsjes - Oud-West,Nawid,f,NaN,9609
14472203,30468760,2018-12-08,f,15,21,f,2969,20181208,8,December,2018,Netherlands,Amsterdam,De Baarsjes - Oud-West,Nawid,f,NaN,9609
14472204,30468760,2018-12-07,f,15,21,f,2969,20181207,7,December,2018,Netherlands,Amsterdam,De Baarsjes - Oud-West,Nawid,f,NaN,9609


In [98]:
Booking_fact_pre = df4.filter(['host_id','id','new_id','country','city','neighbourhood','time_id'])
Booking_fact_pre # Use to upload to dbms

,host_id,id,new_id,country,city,neighbourhood,time_id
0,9662,167183,1,Spain,Madrid,Piovera,20210415
1,9662,167183,1,Spain,Madrid,Piovera,20210416
2,9662,167183,2,Spain,Madrid,Piovera,20210417
3,9662,167183,2,Spain,Madrid,Piovera,20210715
4,9662,167183,2,Spain,Madrid,Piovera,20210716
...,...,...,...,...,...,...,...
14472201,9609,30468760,2969,Netherlands,Amsterdam,De Baarsjes - Oud-West,20181210
14472202,9609,30468760,2969,Netherlands,Amsterdam,De Baarsjes - Oud-West,20181209
14472203,9609,30468760,2969,Netherlands,Amsterdam,De Baarsjes - Oud-West,20181208
14472204,9609,30468760,2969,Netherlands,Amsterdam,De Baarsjes - Oud-West,20181207


In [ ]:
ee

In [99]:
##### Loading data into dbms
import psycopg2 as pg
conn = pg.connect(user = "postgres",
                 password = "BU669",
                 host = "127.0.0.1",
                 port = "5434",
                 database = "CS689_Final_project")
cur = conn.cursor()


In [ ]:
# Create Reviewer_dim table in database
cur.execute("""CREATE TABLE IF NOT EXISTS Reviewer_dim(reviewer_id int Primary Key, reviewer_name varchar,
                comments varchar);""")
conn.commit()


In [ ]:
# Load Reviewr_dim data into DBMS
for index, row in reviewer_dim.iterrows():
    cur.execute("INSERT INTO Reviewer_dim(reviewer_id, reviewer_name, comments) values (%s,%s,%s);",
               (row.reviewer_dim_id, row.reviewer_name, row.comments))
conn.commit()

In [ ]:
# Create Time_dim table in database
cur.execute("""CREATE TABLE IF NOT EXISTS Time_dim(time_id int Primary Key, date date, 
                day int, month varchar, year int);""")
conn.commit()


In [ ]:
# Load time_dim data into DBMS
for index, row in date_dimension_dist.iterrows():
    cur.execute("INSERT INTO time_dim(time_id, date, day, month, year) values (%s,%s,%s,%s,%s);",
               (row.time_id, row.date, row.day, row.month, row.year))
conn.commit()

In [ ]:
# Create Listing_dim table in database
cur.execute("""CREATE TABLE IF NOT EXISTS Listing_dim(listing_dim_id int Primary Key, listing_id int, name varchar,
                property_type varchar, room_type varchar, price int, accommodates varchar, amenities varchar,
                description varchar, effective_timestamp date, expire_timestamp date, current_flag varchar);""")
conn.commit()


In [ ]:
# Load Listing_dim data into DBMS
for index, row in Listing_Distinct_load.iterrows():
    cur.execute("""INSERT INTO Listing_dim(listing_dim_id, listing_id,  name, property_type,
                room_type, price, accommodates, amenities, description, effective_timestamp, 
                expire_timestamp, current_flag ) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);""",
               (row.listing_dim_id, row.id, row.title, row.property_type, row.room_type, row.price,
                row.accommodates, row.amenities, row.description, row.effective_timestamp, 
                row.expire_timestamp, row.current_flag))
conn.commit()

In [111]:
### Dataframe for loading into Listing_dim
Listing_Distinct_load = Listing_Distinct.rename(columns = {'name' : 'title'})


In [ ]:
# # Create Review_fact table in database
cur.execute("""CREATE TABLE IF NOT EXISTS Review_fact(review_id int Primary Key, 
                time_id int References Time_dim (time_id), listing_id int References Listing_dim(listing_dim_id), 
                reviewer_id int References reviewer_dim (reviewer_id), 
                reviews_scores_accuracy int, review_scores_communication int, review_score_rating int);""")
conn.commit()


In [ ]:
# Load Review_fact data into DBMS
for index, row in Review_fact_load_21.iterrows():
    listing_dim_id = int(row.listing_dim_id) if not pd.isna(row.listing_dim_id) else None
    review_scores_accuracy = int(row.review_scores_accuracy) if not pd.isna(row.review_scores_accuracy) else None
    review_scores_communication = int(row.review_scores_communication) if not pd.isna(row.review_scores_communication) else None
    review_scores_rating = int(row.review_scores_rating) if not pd.isna(row.review_scores_rating) else None
    time_id = int(row.time_id) if not pd.isna(row.time_id) else None
    reviewer_dim_id = int(row.reviewer_dim_id) if not pd.isna(row.reviewer_dim_id) else None
    cur.execute("""INSERT INTO Review_fact(review_id, time_id, listing_id, reviewer_id, reviews_scores_accuracy,
                review_scores_communication, review_score_rating) values (%s,%s,%s,%s,%s,%s,%s);""",
               (row.review_dim_id, time_id, listing_dim_id, reviewer_dim_id, review_scores_accuracy,
                review_scores_communication,row.review_scores_rating))
conn.commit()


In [ ]:
# Load host_dim data into DBMS. The host_dim table was already created in DBMS
# host_id, host_name,host_is_superhost, host_response_time 
for index, row in host_distinct.iterrows():
    cur.execute("""INSERT INTO host_dim(host_id,host_name, host_is_superhost, host_response_time)
                values(%s,%s,%s,%s)""",
                (row.host_id, row.host_name, row.host_is_superhost, row.host_response_time))
conn.commit()    

In [100]:
# # Create property_available table in database
cur.execute("""CREATE TABLE IF NOT EXISTS property_available (available_id int Primary Key, 
                available varchar, instant_bookable varchar, minimum_nights int, maximum_nights int);""")
conn.commit()

In [102]:
# Load property_available_df data into DBMS
for index, row in property_available_dim.iterrows():
    cur.execute("""INSERT INTO property_available(available_id,available,instant_bookable,minimum_nights,maximum_nights)
                values(%s,%s,%s,%s,%s)""",
                (row.new_id, row.available, row.instant_bookable, row.minimum_nights, row.maximum_nights))
conn.commit()


In [ ]:
# Load location_df data into DBMS
for index, row in location_distinct_df.iterrows():
    cur.execute("""INSERT INTO location_dim(country,city,neighborhood)
                values(%s,%s,%s)""",
                (row.country, row.city, row.neighbourhood))
conn.commit()

In [106]:
# # Create booking_fact_preparation table in database
cur.execute("""CREATE TABLE IF NOT EXISTS booking_fact_preparation(listing_id_p int, 
                 time_id int References Time_dim (time_id),country varchar, city varchar, neighborhood varchar,
                 available_id int References property_available (available_id), 
                 host_id int References host_dim(host_id) );""")
conn.commit()

In [110]:
# Load Booking_fact_pre data into DBMS
for index, row in Booking_fact_pre.iterrows():
    cur.execute("""INSERT INTO booking_fact_preparation(listing_id_p, time_id,country, city, neighborhood,
                 available_id,host_id)
                values(%s,%s,%s,%s,%s,%s,%s)""",
                (row.id,row.time_id,row.country,row.city,row.neighbourhood,row.new_id,row.host_id))
conn.commit()